#  第三章：编写注意力机制

 在这个notebook中使用的包有：

In [1]:
from importlib.metadata import version
import torch

print("torch version:", version("torch"))

torch version: 2.1.1


 ## 3.1 长序列建模的问题

- 这个部分没有代码。

 ## 3.2 使用注意力机制捕获数据依赖性

- 这个部分没有代码。

 ## 3.3 使用自注意力关注输入的不同部分

 ### 3.3.1 一个简单的自注意力机制，不包含可训练权重

 - 本部分介绍了一个极其简化的自注意力机制版本，它不包含任何可训练的权重。这只是为了说明概念，并不是实际在 transformers 模型中使用的注意力机制。接下来的3.3.2节将扩展这个简单的注意力机制，介绍真正的自注意力机制。
- 假设我们有一个输入序列，从 $x^{(1)}$ 到 $x^{(T)}$。
  - 输入是一段文本（比如句子 "Your journey starts with one step"），它已经被转换成了第二章中描述的标记嵌入形式。
  - 例如，$x^{(1)}$ 是一个 d 维的向量，代表了单词 "Your"，依此类推。
- **目标：** 为输入序列中的每个元素 $x^{(i)}$（从 $x^{(1)}$ 到 $x^{(T)}$，其中 $z$ 和 $x$ 的维度相同）计算上下文向量 $z^{(i)}$。
    - 上下文向量 $z^{(i)}$ 是对输入 $x^{(1)}$ 到 $x^{(T)}$ 的加权平均。
    - 上下文向量是针对特定输入的上下文信息。
      - 我们不使用 $x^{(i)}$ 作为任意输入标记的占位符，而是考虑第二个输入，$x^{(2)}$。
      - 为了具体说明，我们不是用占位符 $z^{(i)}$，而是考虑第二个输出的上下文向量，$z^{(2)}$。
      - 第二个上下文向量 $z^{(2)}$ 是对所有输入 $x^{(1)}$ 到 $x^{(T)}$ 的加权平均，权重是根据第二个输入元素 $x^{(2)}$ 来确定的。这些注意力权重决定了在计算 $z^{(2)}$ 时，每个输入元素对最终加权平均的贡献程度。
    - 简而言之，可以把 $z^{(2)}$ 看作是 $x^{(2)}$ 的一个变体，它不仅包含了 $x^{(2)}$ 的信息，还融合了与当前任务相关的所有其他输入元素的信息。

- 按照惯例，未经归一化的注意力权重被称为**“注意力分数”**，而归一化后的注意力分数（它们的和为1）被称为**“注意力权重”**。

- 注意力权重和上下文向量的计算总结在下面的图表中：

<img src="figures/attention.png" width="600px">

 - 下面的代码逐步展示了上面图表的内容。

<br>

- **步骤 1：** 计算未归一化的注意力分数 $\omega$。
  - 假设我们使用第二个输入标记作为查询，即 $q^{(2)} = x^{(2)}$，我们通过点积来计算未归一化的注意力分数：
    - $\omega_{21} = x^{(1)} \cdot q^{(2)\top}$
    - $\omega_{22} = x^{(2)} \cdot q^{(2)\top}$
    - $\omega_{23} = x^{(3)} \cdot q^{(2)\top}$
    - ...
    - $\omega_{2T} = x^{(T)} \cdot q^{(2)\top}$
  - 在这里，$\omega$ 是希腊字母 "omega"，用来表示未归一化的注意力分数。
    - $\omega_{21}$ 中的下标 "21" 表示输入序列的第2个元素被用作查询，与输入序列的第1个元素进行比较。

<img src="figures/dot-product.png" width="450px">

- 假设我们有以下已经转换成3维向量的输入句子，如第三章所述（为了说明方便，这里使用了一个非常小的嵌入维度，以便在不换行的情况下适应页面）：

In [2]:
import torch

inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

- 我们以输入序列中的第二个元素 $x^{(2)}$ 为例，来计算上下文向量 $z^{(2)}$；在后面的部分，我们将推广这个方法来计算所有的上下文向量。
- 第一步是计算未归一化的注意力分数，通过计算查询 $x^{(2)}$ 与所有其他输入标记之间的点积来实现：

In [3]:
# 从输入序列中取出第二个元素作为查询向量
query = inputs[1]

# 创建一个空的张量来存储注意力分数，其形状与输入序列的批次大小相同
attn_scores_2 = torch.empty(inputs.shape[0])

# 遍历输入序列的每个元素
for i, x_i in enumerate(inputs):
    # 计算当前元素与查询向量的点积作为注意力分数
    # 这里不需要转置，因为假设输入向量是一维的
    attn_scores_2[i] = torch.dot(x_i, query)

# 打印注意力分数
print(attn_scores_2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


- 注：点积实际上是一种简写，它表示将两个向量的对应元素相乘，然后将这些乘积相加求和：

In [4]:
# 初始化结果变量为0
res = 0.

# 遍历输入序列的第一个元素中的每个元素
for idx, element in enumerate(inputs[0]):
    # 将当前元素与查询向量的对应元素相乘，并将结果累加到res
    res += inputs[0][idx] * query[idx]

# 打印手动计算的点积结果
print(res)

# 使用PyTorch的torch.dot函数计算点积，并打印结果
print(torch.dot(inputs[0], query))

tensor(0.9544)
tensor(0.9544)


- **第二步：** 对未归一化的注意力分数（称为“omegas”，用希腊字母 $\omega$ 表示）进行归一化处理，使得它们的总和等于1。
- 这里有一个简单的方法来归一化这些未归一化的注意力分数，以确保它们的总和为1（这是一个常用的做法，有助于理解，并且对训练过程的稳定性至关重要）：

In [5]:
# 使用注意力分数的和对注意力分数进行归一化处理
attn_weights_2_tmp = attn_scores_2 / attn_scores_2.sum()

# 打印归一化后的注意力权重
print("Attention weights:", attn_weights_2_tmp)
# 验证归一化后的注意力权重之和是否为1
print("Sum:", attn_weights_2_tmp.sum())

Attention weights: tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
Sum: tensor(1.0000)


- 然而，在实际应用中，使用softmax函数进行归一化更为常见且推荐，因为它更擅长处理极端值，在训练过程中具有更理想的梯度特性。
- 以下是一个简单的softmax函数实现，它用于缩放，同时归一化向量元素，使得它们的总和为1：

In [6]:
# 定义一个简单的 softmax 函数实现
def softmax_naive(x):
    # 对输入张量 x 的每个元素应用指数函数
    exp_x = torch.exp(x)
    # 计算 exp_x 在指定维度（这里是第一个维度，dim=0）上的和
    sum_exp_x = exp_x.sum(dim=0)
    # 将 exp_x 的每个元素除以它们的和，得到 softmax 结果
    return exp_x / sum_exp_x

# 使用 naive softmax 函数对注意力分数进行归一化
attn_weights_2_naive = softmax_naive(attn_scores_2)

# 打印归一化后的注意力权重
print("Attention weights:", attn_weights_2_naive)
# 验证归一化后的注意力权重之和是否为1
print("Sum:", attn_weights_2_naive.sum())

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


- 上面的简单实现可能会因为输入值过大或过小而导致数值不稳定问题，这主要是因为数值溢出和下溢的问题。
- 因此，在实际应用中，建议使用 PyTorch 提供的 `softmax` 函数实现，它经过高度优化，性能更优：

In [7]:
# 使用 PyTorch 的 softmax 函数对注意力分数进行归一化
# dim=0 表示在第一个维度（通常是特征维度）上进行 softmax 计算
attn_weights_2 = torch.softmax(attn_scores_2, dim=0)

# 打印归一化后的注意力权重
print("Attention weights:", attn_weights_2)
# 验证归一化后的注意力权重之和是否为1
print("Sum:", attn_weights_2.sum())

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


 - **第三步**：通过将嵌入的输入标记 $x^{(i)}$ 与注意力权重相乘，然后将得到的结果向量相加，来计算上下文向量 $z^{(2)}$：

In [8]:
# 选择输入序列中的第二个元素作为查询向量
query = inputs[1]

# 初始化上下文向量，其形状与查询向量相同，初始值为0
context_vec_2 = torch.zeros(query.shape)

# 遍历输入序列中的每个元素
for i, x_i in enumerate(inputs):
    # 累加每个输入元素与其对应的注意力权重的乘积
    context_vec_2 += attn_weights_2[i] * x_i

# 打印计算得到的上下文向量
print(context_vec_2)

tensor([0.4419, 0.6515, 0.5683])


### 3.3.2 计算所有输入标记的注意力权重

#### 推广到所有输入序列标记：

- 在上面的内容中，我们计算了输入2的注意力权重和上下文向量（如下面图表中高亮行所示）。
- 接下来，我们将推广这个计算过程，以计算所有输入标记的注意力权重和上下文向量。

<img src="figures/attention-matrix.png" width="400px">

 - 应用之前的**第一步**，对所有成对的元素进行计算，以得到未归一化的注意力分数矩阵：

In [9]:
# 创建一个 6x6 的零张量，用于存储注意力分数
attn_scores = torch.empty(6, 6)

# 遍历输入序列中的每个元素
for i, x_i in enumerate(inputs):
    # 对于当前的输入元素 x_i，再次遍历整个输入序列
    for j, x_j in enumerate(inputs):
        # 计算 x_i 和 x_j 的点积，作为注意力分数，并存储在 attn_scores 矩阵的对应位置
        attn_scores[i, j] = torch.dot(x_i, x_j)

# 打印完整的注意力分数矩阵
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


- 我们可以通过矩阵乘法更有效地实现上面的计算：

In [10]:
# 使用矩阵乘法计算输入序列的点积矩阵
# inputs @ inputs.T 相当于 inputs 与 inputs 的转置相乘
attn_scores = inputs @ inputs.T

# 打印注意力分数矩阵
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


- 与之前的**第二步**类似，我们对每一行进行归一化，以使每一行的值之和为1：

In [11]:
attn_weights = torch.softmax(attn_scores, dim=1)
print(attn_weights)

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


- 快速验证每一行的值确实之和为1：

In [12]:
# 定义第二行的注意力权重列表
row_2_sum = sum([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
# 打印第二行的和
print("Row 2 sum:", row_2_sum)

# 使用 PyTorch 的 sum 函数沿着指定维度（这里是维度1，即行）计算所有行的和
# attn_weights.sum(dim=1) 会返回一个包含每行和的一维张量
print("All row sums:", attn_weights.sum(dim=1))

Row 2 sum: 1.0
All row sums: tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])


- 应用之前的**第三步**，计算所有上下文向量：

In [13]:
all_context_vecs = attn_weights @ inputs
print(all_context_vecs)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


- 作为合理性检查，之前计算的上下文向量 $z^{(2)} = [0.4419, 0.6515, 0.5683]$ 可以在上面的第二行找到：

In [14]:
print("Previous 2nd context vector:", context_vec_2)

Previous 2nd context vector: tensor([0.4419, 0.6515, 0.5683])


## 3.4 使用可训练权重实现自注意力

### 3.4.1 逐步计算注意力权重

- 在本节中，我们正在实现自注意力机制，这是原始 transformer 架构、GPT模型以及大多数其他流行的大型语言模型（LLMs）中使用的技术。
- 这种自注意力机制也被称为“缩放点积注意力”。
- 总体思路与之前类似：
  - 我们想要计算上下文向量，作为特定输入元素的输入向量的加权和。
  - 为此，我们需要注意力权重。
- 你将看到，与之前介绍的基本注意力机制相比，只有细微的差别：
  - 最显著的区别是引入了在模型训练期间更新的权重矩阵。
  - 这些可训练的权重矩阵至关重要，以便模型（特别是模型内的注意力模块）能够学习产生 "good" 上下文向量。

- 在逐步实现自注意力机制的过程中，我们将首先引入三个训练权重矩阵 $W_q$、$W_k$ 和 $W_v$。
- 这三个矩阵用于通过矩阵乘法将嵌入的输入标记 $x^{(i)}$ 投影到查询、键和值向量：

  - 查询向量：$q^{(i)} = W_q \cdot x^{(i)}$
  - 键向量：$k^{(i)} = W_k \cdot x^{(i)}$
  - 值向量：$v^{(i)} = W_v \cdot x^{(i)}$

<img src="figures/weight-selfattn-1.png" width="600px">

- 输入 $x$ 的嵌入维度和查询向量 $q$ 的维度可以相同也可以不同，这取决于模型的设计和具体实现。
- 在 GPT 模型中，输入和输出的维度通常是相同的，但为了更好地说明计算过程，我们在这里选择不同的输入和输出维度：

In [15]:
# 获取输入序列中的第二个元素作为特定的输入向量
x_2 = inputs[1]

# 获取输入张量的嵌入维度，这里假设输入向量的维度为3
d_in = inputs.shape[1]

# 设置输出嵌入的维度，这里假设输出向量的维度为2
d_out = 2

- 下面，我们初始化这三个权重矩阵；请注意，我们设置了 `requires_grad=False` 以减少输出中的混乱，这是为了说明目的。但是，如果我们要在模型训练中使用这些权重矩阵，我们会将 `requires_grad` 设置为 `True`，以便在模型训练过程中更新这些矩阵。

In [16]:
# 设置随机种子以确保结果的可重复性
torch.manual_seed(123)

# 创建查询权重矩阵，形状为 (d_in, d_out)，并且设置 requires_grad=False 表示这些权重在训练过程中不会更新
W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

# 创建键权重矩阵，形状和 W_query 相同，同样设置 requires_grad=False
W_key = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

# 创建值权重矩阵，形状和 W_query 相同，同样设置 requires_grad=False
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

- 接下来，我们计算查询、键和值向量：

In [17]:
# 使用查询权重矩阵 W_query 将第二个输入元素 x_2 投影到查询空间
query_2 = x_2 @ W_query  # 使用 @ 运算符进行矩阵乘法

# 使用键权重矩阵 W_key 将第二个输入元素 x_2 投影到键空间
key_2 = x_2 @ W_key

# 使用值权重矩阵 W_value 将第二个输入元素 x_2 投影到值空间
value_2 = x_2 @ W_value

# 打印计算得到的查询向量
print(query_2)

tensor([0.4306, 1.4551])


- 从下面的结果可以看出，我们成功地将6个输入标记从一个3维空间投影到了一个2维嵌入空间：

In [18]:
# 使用键权重矩阵 W_key 将输入序列 inputs 投影到键空间
keys = inputs @ W_key

# 使用值权重矩阵 W_value 将输入序列 inputs 投影到值空间
values = inputs @ W_value

# 打印键向量的形状
print("keys.shape:", keys.shape)

# 打印值向量的形状
print("values.shape:", values.shape)

keys.shape: torch.Size([6, 2])
values.shape: torch.Size([6, 2])


- 在下一步，即**第二步**中，我们通过计算查询向量与每个键向量之间的点积来计算未归一化的注意力分数：

<img src="figures/weight-selfattn-2.png" width="600px">

In [19]:
# 从键张量中提取第二个键向量，对应于输入序列中的第二个元素
keys_2 = keys[1]  # Python 中的索引是从 0 开始的

# 计算查询向量 query_2 和键向量 keys_2 之间的点积，得到注意力分数
attn_score_22 = query_2.dot(keys_2)

# 打印计算得到的注意力分数
print(attn_score_22)

tensor(1.8524)


- 由于我们有6个输入，对于给定的查询向量，我们得到了6个注意力分数：

In [20]:
# 使用查询向量 query_2 和所有键向量 keys 的转置（keys.T）进行矩阵乘法，得到所有注意力分数
attn_scores_2 = query_2 @ keys.T

# 打印所有注意力分数
print(attn_scores_2)

tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])


<img src="figures/weight-selfattn-3.png" width="600px">

- 接下来，在**第三步**中，我们使用之前提到的 softmax 函数来计算注意力权重（归一化的注意力分数，它们的总和为1）。
- 与之前不同的是，我们现在通过除以嵌入维度的平方根，$\sqrt{d_k}$（即 `d_k**0.5`），来缩放注意力分数：

In [21]:
# 获取键向量的维度，即每个键向量的维度大小
d_k = keys.shape[1]

# 使用 softmax 函数对注意力分数进行归一化处理
# attn_scores_2 / d_k**0.5 是缩放点积分数，防止过大的点积值导致 softmax 梯度过小
# dim=-1 表示沿着最后一个维度（即注意力分数的维度）进行 softmax 计算
attn_weights_2 = torch.softmax(attn_scores_2 / d_k**0.5, dim=-1)

# 打印归一化后的注意力权重
print(attn_weights_2)

tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])


<img src="figures/weight-selfattn-4.png" width="600px">

- 在**第四步**中，我们现在计算输入查询向量2的上下文向量：

In [22]:
# 使用归一化的注意力权重 attn_weights_2 和值向量 values 进行矩阵乘法，得到上下文向量
context_vec_2 = attn_weights_2 @ values

# 打印计算得到的上下文向量
print(context_vec_2)

tensor([0.3061, 0.8210])


### 3.4.2 实现一个紧凑的 SelfAttention 类

- 将所有内容整合起来，我们可以按照以下方式实现自注意力机制：

In [23]:
import torch.nn as nn

# 定义自注意力模块
class SelfAttention_v1(nn.Module):
    def __init__(self, d_in, d_out):
        # 调用父类构造函数
        super().__init__()
        # 设置输出维度
        self.d_out = d_out
        # 初始化查询、键和值的权重矩阵，这些矩阵是可训练的
        self.W_query = nn.Parameter(torch.rand(d_in, d_out))
        self.W_key = nn.Parameter(torch.rand(d_in, d_out))
        self.W_value = nn.Parameter(torch.rand(d_in, d_out))

    def forward(self, x):
        # 使用权重矩阵将输入 x 投影到查询、键和值空间
        keys = x @ self.W_key
        queries = x @ self.W_query
        values = x @ self.W_value
        
        # 计算注意力分数（未归一化）
        attn_scores = queries @ keys.T  # omega
        
        # 使用 softmax 函数和缩放因子归一化注意力分数
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)

        # 使用归一化的注意力权重和值向量计算上下文向量
        context_vec = attn_weights @ values
        return context_vec

# 设置随机种子以确保结果的可重复性
torch.manual_seed(123)
# 创建 SelfAttention_v1 实例
sa_v1 = SelfAttention_v1(d_in, d_out)
# 使用输入数据 inputs 进行前向传播，并打印结果
print(sa_v1(inputs))

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


- 我们可以使用 PyTorch 的线性层 `nn.Linear` 来简化上述实现，如果我们关闭偏置单元，它们就等同于矩阵乘法。
- 使用 `nn.Linear` 而不是我们手动创建的 `nn.Parameter(torch.rand(...))` 方法的另一个重要优势是，`nn.Linear` 自带了一种优选的权重初始化方案，这有助于实现更稳定的模型训练。

In [24]:
import torch.nn as nn

# 定义自注意力模块的第二个版本
class SelfAttention_v2(nn.Module):
    def __init__(self, d_in, d_out, qkv_bias=False):
        # 调用父类构造函数
        super().__init__()
        # 设置输出维度
        self.d_out = d_out
        # 初始化查询、键和值的线性层，可以选择是否包含偏置项
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

    def forward(self, x):
        # 使用线性层将输入 x 投影到查询、键和值空间
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)
        
        # 计算注意力分数（未归一化）
        attn_scores = queries @ keys.T
        
        # 使用 softmax 函数和缩放因子归一化注意力分数
        # 注意这里的 dim=1，表示沿着键向量的维度进行归一化
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=1)

        # 使用归一化的注意力权重和值向量计算上下文向量
        context_vec = attn_weights @ values
        return context_vec

# 设置随机种子以确保结果的可重复性
torch.manual_seed(789)
# 创建 SelfAttention_v2 实例
sa_v2 = SelfAttention_v2(d_in, d_out)
# 使用输入数据 inputs 进行前向传播，并打印结果
print(sa_v2(inputs))

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)


- 请注意，`SelfAttention_v1` 和 `SelfAttention_v2` 会产生不同的输出，因为它们使用了不同的初始权重矩阵。

## 3.5 遮蔽下文信息的注意力机制

### 3.5.1 使用因果注意力掩码

- 在本节中，我们将前面的自注意力机制转换为因果自注意力机制。

- 因果自注意力机制的核心目标是，确保模型对序列中某个位置的预测只依赖于前面位置的已知输出（也就是上文），而不依赖于未来位置（也就是下文）。也就是说，确保每一个词的预测只应该依赖于前面的词。

- 为了实现这一点，对于每个给定的词，我们屏蔽掉未来的词（即输入文本中在当前词之后的词）。

<img src="figures/masked.png" width="600px">

- 为了说明和实现因果自注意力机制，让我们使用上一节的注意力分数和权重进行操作。

In [25]:
# 使用上一节中 SelfAttention_V2 的 query 和 key 的权重矩阵
queries = sa_v2.W_query(inputs)
keys = sa_v2.W_key(inputs) 
attn_scores = queries @ keys.T
# 此处的注意力权重和上一节中的一致
attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=1)
print(attn_weights)

tensor([[0.1921, 0.1646, 0.1652, 0.1550, 0.1721, 0.1510],
        [0.2041, 0.1659, 0.1662, 0.1496, 0.1665, 0.1477],
        [0.2036, 0.1659, 0.1662, 0.1498, 0.1664, 0.1480],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.1661, 0.1564],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.1585],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


- 屏蔽未来的注意力权重最简单的方法是通过 PyTorch 的 tril 函数创建一个掩码，主对角线（包括对角线本身）以下的元素设置为 1，主对角线以上的元素设置为 0：

In [26]:
# 我们创建的掩码形状应该和注意力权重矩阵的形状一致，以一一对应
block_size = attn_scores.shape[0]
# tril 方法会创建一个下三角矩阵
mask_simple = torch.tril(torch.ones(block_size, block_size))
print(mask_simple)

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])


- 然后，我们可以将注意力权重与这个掩码相乘，从而将对角线以上的注意力分数归零：

In [27]:
masked_simple = attn_weights*mask_simple
print(masked_simple)

tensor([[0.1921, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2041, 0.1659, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2036, 0.1659, 0.1662, 0.0000, 0.0000, 0.0000],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.0000, 0.0000],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<MulBackward0>)


- 然而，如果像上文一样，在 softmax 之后再应用掩码，它会破坏 softmax 创建的概率分布。Softmax将确保所有输出值的总和为 1，但由于我们将部分输出值置为了 0，这将导致输出值总和发生变化。

- 因此，在 softmax 之后进行掩码处理将需要重新对输出进行归一化，使其总和再次为 1。但是，这使得过程变得复杂，并可能导致意想不到的效果。

- 为了确保输出值的总和为 1，我们可以将权重矩阵进行如下的归一化：

In [28]:
# dim = 1 表示按行求和
row_sums = masked_simple.sum(dim=1, keepdim=True)
masked_simple_norm = masked_simple / row_sums
print(masked_simple_norm)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<DivBackward0>)


- 尽管我们现在在技术上已经完成了因果注意力机制，但还有一些实现上述相同效果的更有效的方法。

- 例如，我们可以在未归一化的注意力分数进入 softmax 函数之前，用负无穷大掩盖对角线以上的部分，而不是将对角线以上的注意力权重归零并重新归一化结果。

In [29]:
# 也就是说，通过将掩码从 0 修改为 -inf，可以将遮蔽操作提到 softmax 之前
mask = torch.triu(torch.ones(block_size, block_size), diagonal=1)
masked = attn_scores.masked_fill(mask.bool(), -torch.inf)
print(masked)

tensor([[0.2899,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.4656, 0.1723,   -inf,   -inf,   -inf,   -inf],
        [0.4594, 0.1703, 0.1731,   -inf,   -inf,   -inf],
        [0.2642, 0.1024, 0.1036, 0.0186,   -inf,   -inf],
        [0.2183, 0.0874, 0.0882, 0.0177, 0.0786,   -inf],
        [0.3408, 0.1270, 0.1290, 0.0198, 0.1290, 0.0078]],
       grad_fn=<MaskedFillBackward0>)


- 正如我们所见，接下来我们再让注意力矩阵通过 softmax，就可以将每行之和都重新变回 1：

In [30]:
attn_weights = torch.softmax(masked / keys.shape[-1]**0.5, dim=1)
print(attn_weights)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


### 3.5.2 通过 dropout 来实现额外注意力权重的掩码

- 此外，我们还可以在训练过程中应用 dropout 来减少过拟合。

- dropout 可以应用于例如下列例子的多个地方：
    - 计算注意力权重后；
    - 将注意力权重与值向量相乘后。

- 在这里，我们将在计算注意力权重后应用 dropout 掩码，因为这种情况更为常见。

- 此外，在这个特定的例子中，我们使用了 50% 的 dropout 率，这意味着随机屏蔽一半的注意力权重。（当我们稍后训练 GPT 模型时，我们将使用较低的 dropout 率，例如 0.1 或 0.2。）

<img src="figures/dropout.png" width="500px">

- 注意，如果我们应用 0.5 的 dropout 率，那么未被屏蔽的值将按照 1/0.5 = 2 的比例进行相应缩放。

In [30]:
# 随便设置一个随机数种子
torch.manual_seed(123)
dropout = torch.nn.Dropout(0.5) # 设置 50% 的 Dropout 比例
example = torch.ones(6, 6) # 创建一个全 1 矩阵作为示例

print(dropout(example))

tensor([[2., 2., 2., 2., 2., 2.],
        [0., 2., 0., 0., 0., 0.],
        [0., 0., 2., 0., 2., 0.],
        [2., 2., 0., 0., 0., 2.],
        [2., 0., 0., 0., 0., 2.],
        [0., 2., 0., 0., 0., 0.]])


In [31]:
torch.manual_seed(123)
# 对注意力权重进行 dropout
print(dropout(attn_weights))

tensor([[0.3843, 0.3293, 0.3303, 0.3100, 0.3442, 0.3019],
        [0.0000, 0.3318, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.3325, 0.0000, 0.3328, 0.0000],
        [0.3738, 0.3334, 0.0000, 0.0000, 0.0000, 0.3128],
        [0.3661, 0.0000, 0.0000, 0.0000, 0.0000, 0.3169],
        [0.0000, 0.3327, 0.0000, 0.0000, 0.0000, 0.0000]],
       grad_fn=<MulBackward0>)


## 3.5.3 实现一个因果自注意类

- 现在，我们已经准备好实现一个包含 dropout 的因果自注意力类。

- 我们还需要实现处理由多个输入组成的一批样本的代码，以便我们的 CausalAttention 类支持我们在第2章中实现的 dataloader 产生的批量输出。

- 为了简化，为了模拟这样的批量输入，我们复制输入文本示例：

In [32]:
batch = torch.stack((inputs, inputs), dim=0)
print(batch.shape) # 2个输入，每个输入有 6个 token，每个 token 的维度为 3

torch.Size([2, 6, 3])


In [33]:
# 定义一个带 dropout 的因果自注意力层
class CausalAttention(nn.Module):

    def __init__(self, d_in, d_out, block_size, dropout, qkv_bias=False):
        '''
        构造函数，输入参数如下：
        d_in: 输入的维度
        d_out: 输出的维度
        block_size: 注意力权重矩阵的大小
        dropout: dropout 比例
        qkv_bias: 是否对 query、key 和 value 加偏置
        '''
        super().__init__()
        self.d_out = d_out
        # 根据前文，每一个权重矩阵都是 d_in x d_out 的线性层
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        # 一个 dropout 层
        self.dropout = nn.Dropout(dropout) 
        # 一个掩码矩阵，下三角为 1，其余为 0
        self.register_buffer('mask', torch.triu(torch.ones(block_size, block_size), diagonal=1)) # New

    def forward(self, x):
        '''
        前向传播函数，输入参数为 x，维度为 b x num_tokens x d_in，输出维度为 b x num_tokens x d_out
        '''
        b, num_tokens, d_in = x.shape
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)
        # transpose 是为了实现矩阵乘法
        attn_scores = queries @ keys.transpose(1, 2)
        # 即上文说过的，将掩码从 0 修改为 -inf，再进行遮蔽操作
        attn_scores.masked_fill_(  # New, _ ops are in-place
            self.mask.bool()[:num_tokens, :num_tokens], -torch.inf)
        # 经过 softmax 
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=1)
        # 进行 dropout
        attn_weights = self.dropout(attn_weights) # New
        # 得到最后结果
        context_vec = attn_weights @ values
        return context_vec

# 实验一下
torch.manual_seed(123)

block_size = batch.shape[1]
ca = CausalAttention(d_in, d_out, block_size, 0.0)

context_vecs = ca(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[-0.0844,  0.0414],
         [-0.2264, -0.0039],
         [-0.4163, -0.0564],
         [-0.5014, -0.1011],
         [-0.7754, -0.1867],
         [-1.1632, -0.3303]],

        [[-0.0844,  0.0414],
         [-0.2264, -0.0039],
         [-0.4163, -0.0564],
         [-0.5014, -0.1011],
         [-0.7754, -0.1867],
         [-1.1632, -0.3303]]], grad_fn=<UnsafeViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])


- 注意 dropout 只在训练阶段被使用，在推理阶段是不使用的

## 3.6 将单头注意力扩展到多头

### 3.6.1 直接将多个单头注意力层堆积起来

- 下图是之前提到过的自注意力的总结（为了简便起见，因果注意力掩码和 dropout 并没有展示） 

- 也被称之为单头注意力:

<img src="figures/single-head.png" width="600px">

- 我们可以简单地将多个单头注意力层堆积在一起实现多头注意力层:

<img src="figures/multi-head.png" width="600px">

- 多头注意力机制的主要思想是使用不同的、已学习的权重矩阵，多次（并行）运行注意力机制。这使得模型能够在不同位置的不同表示子空间中联合关注信息。

In [34]:
# 定义一个多头注意力层
class MultiHeadAttentionWrapper(nn.Module):

    def __init__(self, d_in, d_out, block_size, dropout, num_heads, qkv_bias=False):
        super().__init__()
            # 将 num_heads 个单头注意力层组合在一起来实现多头
        self.heads = nn.ModuleList(
            [CausalAttention(d_in, d_out, block_size, dropout, qkv_bias) 
             for _ in range(num_heads)]
        )

    def forward(self, x):
        # 前向计算时将多个头的输出拼接在一起
        return torch.cat([head(x) for head in self.heads], dim=-1)


# 实验一下
torch.manual_seed(123)

block_size = batch.shape[1] # token 数量
d_in, d_out = 3, 2
mha = MultiHeadAttentionWrapper(d_in, d_out, block_size, 0.0, num_heads=2)

context_vecs = mha(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[-0.0844,  0.0414,  0.0766,  0.0171],
         [-0.2264, -0.0039,  0.2143,  0.1185],
         [-0.4163, -0.0564,  0.3878,  0.2453],
         [-0.5014, -0.1011,  0.4992,  0.3401],
         [-0.7754, -0.1867,  0.7387,  0.4868],
         [-1.1632, -0.3303,  1.1224,  0.8460]],

        [[-0.0844,  0.0414,  0.0766,  0.0171],
         [-0.2264, -0.0039,  0.2143,  0.1185],
         [-0.4163, -0.0564,  0.3878,  0.2453],
         [-0.5014, -0.1011,  0.4992,  0.3401],
         [-0.7754, -0.1867,  0.7387,  0.4868],
         [-1.1632, -0.3303,  1.1224,  0.8460]]], grad_fn=<CatBackward0>)
context_vecs.shape: torch.Size([2, 6, 4])


- 在上面的实现中，嵌入维度是4，因为我们为 key、query、value 都设置了 d_out=2 作为嵌入维度。由于我们有2个注意力头，因此输出嵌入维度为 2*2=4。

- 如果我们想要输出维度为2，就像早期的单头注意力那样，我们可以将投影维度 d_out 更改为1：

In [36]:
torch.manual_seed(123)

d_out = 1
mha = MultiHeadAttentionWrapper(d_in, d_out, block_size, 0.0, num_heads=2)

context_vecs = mha(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[-9.1476e-02,  3.4164e-02],
         [-2.6796e-01, -1.3427e-03],
         [-4.8421e-01, -4.8909e-02],
         [-6.4808e-01, -1.0625e-01],
         [-8.8380e-01, -1.7140e-01],
         [-1.4744e+00, -3.4327e-01]],

        [[-9.1476e-02,  3.4164e-02],
         [-2.6796e-01, -1.3427e-03],
         [-4.8421e-01, -4.8909e-02],
         [-6.4808e-01, -1.0625e-01],
         [-8.8380e-01, -1.7140e-01],
         [-1.4744e+00, -3.4327e-01]]], grad_fn=<CatBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])


### 3.6.2 通过权重分割实现多头注意力

- 尽管上述是多头注意力最直观且功能完整的实现（将早期的单头注意力 CausalAttention 实现封装在内），但我们也可以编写一个名为MultiHeadAttention 的独立类来实现相同的功能。

- 对于这个独立的 MultiHeadAttention 类，我们不会将单个注意力头连接在一起。相反，我们创建单个的 W_query、W_key 和 W_value 权重矩阵，然后将它们拆分为每个注意力头的独立矩阵：

In [35]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, block_size, dropout, num_heads, qkv_bias=False):
        super().__init__()
        # 因为要对权重矩阵按注意力头数进行拆分，所有输出维度必须是头数的整数倍
        assert d_out % num_heads == 0, "d_out must be divisible by n_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        # head_dim 就是拆分之后每个头应该输出的维度
        self.head_dim = d_out // num_heads 

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        self.dropout = nn.Dropout(dropout)
        self.register_buffer('mask', torch.triu(torch.ones(block_size, block_size), diagonal=1))

    def forward(self, x):
        b, num_tokens, d_in = x.shape

        # 形状为 (b, num_tokens, d_out)
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        # 我们可以通过增加一个 num_heads 的维度来将矩阵分割到每个头
        # 维度变化: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim) 
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # 转置一下: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # 计算注意力权重
        # 基于矩阵乘法，简单地实现各个头的并行计算
        attn_scores = queries @ keys.transpose(2, 3) 
        # 一般来说我们会将掩码矩阵转化为 bool 值并基于序列的长度进行截断
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]
        # 需要将掩码矩阵 unsqueeze 两次，也就是增加两个维度，才能让掩码矩阵的维度和注意力权重对应上
        mask_unsqueezed = mask_bool.unsqueeze(0).unsqueeze(0)
        # 使用掩码矩阵来进行遮蔽
        attn_scores.masked_fill_(mask_unsqueezed, -torch.inf)
        
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # 形状: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2) 
        
        # 将多个头的输出重新组合回去 self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec) # optional projection

        return context_vec

# 试验一下
torch.manual_seed(123)

batch_size, block_size, d_in = batch.shape
d_out = 2
mha = MultiHeadAttention(d_in, d_out, block_size, 0.0, num_heads=2)

context_vecs = mha(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]],

        [[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]]], grad_fn=<ViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])


- 请注意，以上内容实际上是 MultiHeadAttentionWrapper 的一个更高效的改写版。
- 由于随机权重初始化的差异，最终的输出结果看起来有些不同，但两者都是完全可以使用的实现，将在后续章节中实现的GPT类中使用。
- 此外，我们在上述 MultiHeadAttention 类中添加了一个线性投影层(self.out_proj)。这只是一个不会改变维度的线性变换。在LLM实现中使用这样的投影层是一种标准惯例，但并非严格必要（最近的研究表明，它可以被移除而不会影响建模性能；见本章末尾的进一步阅读部分）

- 如果你对更复杂、高效的多头注意力实现感兴趣，你可以考虑使用 PyTorch 的 [`torch.nn.MultiheadAttention`](https://pytorch.org/docs/stable/generated/torch.nn.MultiheadAttention.html) 类。

- 上述实现可能看上去有一点复杂，让我们来看一下，当运行 `attn_scores = queries @ keys.transpose(2, 3)` 时会发生什么：

In [38]:
# (b, num_heads, num_tokens, head_dim) = (1, 2, 3, 4)
a = torch.tensor([[[[0.2745, 0.6584, 0.2775, 0.8573],
                    [0.8993, 0.0390, 0.9268, 0.7388],
                    [0.7179, 0.7058, 0.9156, 0.4340]],

                   [[0.0772, 0.3565, 0.1479, 0.5331],
                    [0.4066, 0.2318, 0.4545, 0.9737],
                    [0.4606, 0.5159, 0.4220, 0.5786]]]])

print(a @ a.transpose(2, 3))

tensor([[[[1.3208, 1.1631, 1.2879],
          [1.1631, 2.2150, 1.8424],
          [1.2879, 1.8424, 2.0402]],

         [[0.4391, 0.7003, 0.5903],
          [0.7003, 1.3737, 1.0620],
          [0.5903, 1.0620, 0.9912]]]])


- 在这种情况下，PyTorch 中的矩阵乘法实现将处理 4 维输入张量，以便在最后的两个维度（num_tokens，head_dim）之间进行矩阵乘法，然后针对各个头重复进行。

- 例如，上述内容成为了一种单独计算每个头的更紧凑的矩阵乘法：

In [39]:
first_head = a[0, 0, :, :]
first_res = first_head @ first_head.T
print("First head:\n", first_res)

second_head = a[0, 1, :, :]
second_res = second_head @ second_head.T
print("\nSecond head:\n", second_res)

First head:
 tensor([[1.3208, 1.1631, 1.2879],
        [1.1631, 2.2150, 1.8424],
        [1.2879, 1.8424, 2.0402]])

Second head:
 tensor([[0.4391, 0.7003, 0.5903],
        [0.7003, 1.3737, 1.0620],
        [0.5903, 1.0620, 0.9912]])


In [40]:
block_size = 1024
d_in, d_out = 768, 768
num_heads = 12

mha = MultiHeadAttention(d_in, d_out, block_size, 0.0, num_heads)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

count_parameters(mha)

2360064

# 总结与收获

- 你可以查看 [./multihead-attention.ipynb](./multihead-attention.ipynb) 代码 Notebook，这是 DataLoader（第2章）的简洁版本，加上我们在本章中实现的多头注意力类，我们将在后续章节中训练 GPT 模型时使用它。